# Implementación de algoritmo ID3 con atributos continuos.
## Laboratorio 2022 - Tarea 2

### Grupo 27 ###

Rafael Castelli, CI: 5.079.391-2

Alexis Baladón, CI: 5.574.612-4

Ignacio Viscardi, CI: 5.066.666-2







*Aprendizaje Automático*

*Facultad de IngenierÍa. Universidad de la República*

*Montevideo, Uruguay*

# 1. Objetivo

El objetivo de esta tarea es un implementar y evaluar la variación de atributos continuos del algoritmo ID3.

# 2. Diseño




La finalidad de esta sección es introducir las principales decisiones de diseño tomadas para la implementación del agente y el algoritmo de entrenamiento.

## 2.1 Decisiones arquitectónicas

Para la elaboración del obligatorio se implementa el algoritmo ID3 en una variante (dentro de otras posibles) que permitan el manejo de atributos continuos. Como referencia para su implementación se toma el algoritmo planteado por Mitchell [1].


Particularmente se utiliza la ganancia de información como medida para seleccionar los mejores atributos en cada paso recursivo del algoritmo, como es planteado originalmente.


Debido al alcance del laboratorio, se implementan arboles de decisión unicamente para categorías binarias. Extender el algoritmo para categorías discretas finitas arbitrarias implica reescribir algunas funciones auxiliares como el calculo de la entropía o la selección de umbrales para atributos continuos, pero la lógica principal del algoritmo se mantiene igual. Luego, se decide no abstraer el algoritmo a categorías no binarias para evitar lógica “satélite” al problema y poder centrar el esfuerzo en el algoritmo ID3.


Por motivos de simplicidad y reproducibilidad de los experimentos se evitan tomar decisiones estocásticas, como al elegir entre atributos candidatos con mejor ganancia de información. En particular, se prefieren atributos discretos ante atributos continuos y el primer atributo dentro de su categoría (discretos o continuos) según el orden que se define al llamar al algoritmo.


Para el manejo de atributos continuos se utiliza el algoritmo planteado por Mitchell [2] donde un atributo continuo divide al dataset S según un umbral u tal que $S = \{x \in S | x <= u\} \cup \{x \in  S | x > u\}$.



## 2.2 Algoritmo

Para la selección del umbral:


	Sea a el atributo continuo al que se le quiere calcular el umbral optimo para dividir el dataset según dicho atributo.
	Sea c la categoría de una instancia del dataset. 
    Sea C(x) la categoría de la instancia x del dataset.    
	Sea S el dataset.
	Sea U = {}

	Se ordena S de forma creciente según a.
	Se elijen los pares de ejemplos (x,y) adyacentes en el dataset ordenado tales que C(x) != C(y) 
	Para cada par (x,y) en las condiciones anteriores:
		U ← (x + y)/2
	Obtener m,M los valores mínimos y máximos del atributo a en el dataset S.
	U ← U – {m, M}  
	retornar U

Luego, se computa el umbral optimo de la siguiente forma:

	Para cada u ∈ U:
		S_menor, S_mayor = dividir_dataset(S,u)
		ganancia_información(S, S_menor, S_mayor)
	retornar el primer u que maximice la ganancia de información

Este procedimiento se repite para cada atributo continuo y se elige el atributo continuo cuyo umbral optimo tenga mayor ganancia de información.

Posteriormente, se computa la ganancia de información para cada atributo discreto, siguiendo el algoritmo “tradicional” y se compara la ganancia de información del mejor atributo discreto con la ganancia de información del mejor atributo continuo y se elige el mejor de ambos.


Dicha forma de elegir atributos óptimos en cada paso del algoritmo (elegir el optimo continuo, el optimo discreto y después “desempatar”) permite separar en funciones distintas el calculo de cada “tipo” de atributo y por tanto da una mayor modularidad al código y posibilidad de evolución.



Otras diferencias respecto al algoritmo descripto en la bibliografía corresponden a que al algoritmo ID3 implementado se le debe pasar el dominio de cada atributo discreto (los atributos continuos se asume que pueden tomar cualquier real) e indicar en la lista de atributos, cuales son continuos y cuales son discretos.


Esto tiene como motivos evitar tener que implementar lógicas que calculen o infieran el dominio de los atributos y que categoricen (según algún criterio) a los atributos como discretos o continuos, dentro del algoritmo de ID3. 
Luego, en caso de implementar dicha lógica, se debe hacer por fuera del algoritmo que genera el árbol de decisión, por ejemplo en el preprocesado de datos.


En el caso concreto del laboratorio, se opta por no desviar el foco de atención hacia algoritmos y métodos “secundarios” respecto a ID3 y se declaran en el preprocesado de datos.

El dominio de los atributos se puede obtener trivialmente con la librería pandas, y la distinción entre atributos continuos se hace por inspección directa del dataset, así como del numero distinto de valores de los atributos en el dataset. Particularmente, el dominio de un atributo discreto queda dado por todos los valores no nulos (valores como smoking_status = “Unknown” se consideran nulos) que toda en el dataset. 

Los atributos continuos considerados son:
- age,
- avg_glucose_level
- bmi



Otra decisión de diseño tomada corresponde al manejo de atributos nulos. 
La discucion se puede separar en las decisiones tomadas para atributos nulos en el conjunto de entrenamiento y para atributos nulos en la categorización de nuevas instancias por parte del modelo.


Nuevamente,  el reconocimiento de valores nulos se da por el reconocimiento de ciertos “tokens” que se deducen por inspección directa del dataset. De esta forma, “Unknown” es el token para un dato nulo en el atributo “smoking_status”.


Para el entrenamiento del algoritmo se decide solo utilizar instancias que tengan todos sus atributos definidos, requiriendo entonces prepocesar el dataset de entrenamiento antes de pasárselo al algoritmo. 


Esto contrasta con otras alternativas, particularmente las presentadas en la bibliografía, donde se extiende ID3 para lidiar con atributos con valores faltantes. Se opta por el método descripto en el párrafo anterior para simplificar la lógica, aunque consiste en una extensión interesante con la que se podría experimentar como potencial mejora. 


Debido a restricciones temporales, no fue posible implementar dicha extensión, pero como se vera posteriormente, al comparar la solución presentada con la de librerías estándares como sklearn, al menos para el caso y dataset concretos del laboratorio, no parece que esta simplificación impacte de forma significativa en el rendimiento del algoritmo.


Por otro lado, la alternativa de determinar valores faltantes en el mismo algoritmo de ID3 con la que se intento experimentar sufría de tiempos de computo mucho mayores respecto a la alternativa presentada en la solución que no parecen justificarse cuando se compara el rendimiento del algoritmo contra el de la librería sklearn. Es decir, la implementación de dicha extensión que se propuso no era eficiente en tiempo de computo ni uso de memoria y la implementación final entregada (donde los atributos nulos se tratan antes de pasárselos a ID3) contenía una calidad aceptable respecto a librerías estándares.


Respecto a la clasificación de instancias con valores nulos, se crea una clase “TreeWrapper”, que dado el árbol generado por ID3 (de tipo “Tree”), que no acepta valores nulos para clasificación, le agrega metainformación generada a partir del preprocesado del conjunto de entrenamiento y la utiliza para darle un valor a los atributos con valores nulos antes de pasárselos al árbol de ID3 quien realmente asigna una categoría a una instancia.
Por ejemplo, si para el preprocesado del dataset, se reemplazan todos los valores nulos de “smoking_status” por el valor mas común en el dataset de entrenamiento (sea x dicho valor), entonces “TreeWrapper”, dada una instancia nueva que se quiera clasificar, con valor “smoking_status” nulo, la reemplaza por otra instancia idéntica a la original a menos del valor “smoking_status” que ahora vale x. Luego, dicha nueva instancia se pasa al árbol de ID3 para ser clasificada.


Por ultimo, se deciden no aplicar técnicas para evitar el sobre ajuste como “Reduced error pruning” [3]. Como se vera posteriormente, el mayor problema del algoritmo, al menos para este dataset especifico, consiste en el des balance que existe en el dataset dado respecto a la clase objetivo. Además, como se ilustrará, para este caso concreto, no incluir dichas técnicas no parece repercutir significativamente en el rendimiento del algoritmo.
No obstante, dado que los arboles de decisión tienden a sobre ajustar, esta parecería (en términos generales) ser la principal extensión que debería incorporarse al algoritmo para mejorarlo.


## 2.3 Preprocesado de datos:

Como se indico anteriormente, se preprocesan los datos de entrenamiento y los de prueba, en estos últimos particularmente para evitar atributos nulos (cabe destacar que dado el comportamiento de “TreeWrapper” y la técnica de preprocesado aplicadas, resulta equivalente preprocesar los atributos nulos en el conjunto de prueba o pasárselos directamente al algoritmo).


El primer  prepocesado al que son sometidos los datos consiste en la eliminación del atributo “id” y  la transformación a valores reales. (Si bien esto se hace antes de dividir el dataset, es meramente por una cuestión de eficiencia computacional y no va en contra de la metodología).
El atributo id se elimina pues consiste en un clasificador perfecto porque no existen dos tuplas con el mismo valor de id.

 Eliminando el atributo se evita que el algoritmo “memorice” y se reduce el sobre ajuste. Por otro lado, el id siendo un número presuntamente asignado al azar, no tiene relación ninguna (o no debería, mas allá de casualidades en los datos) con saber si una persona tuvo o no un derrame cerebral, es decir no tiene relación ninguna con la clase que se quiere predecir. Por otro lado, los demás atributos parecen estar relacionados con la salud del individuo, ya sea mental o física, que podrían favorecer al padecimiento de un derrame cerebral.


Por otro lado, se transforman todos los valores a reales por compatibilidad con la librería sklearn. El algoritmo implementado acepta atributos categóricos de un tipo arbitrario, pero sklearn solo acepta valores numéricos. Luego, con el motivo de comparar los algoritmos se transforman todos los atributos para tener un numérico. Como nota al margen, es de esperar que trabajar con valores reales permita mejores tiempos de ejecución y uso de memoria que trabajar con tipos arbitrarios como String.


Los atributos discretos son los únicos que potencialmente deben transformarse a reales. Para ello, se le asigna a cada valor de un atributo continuo un numero natural distinto.     


A modo de ejemplo:


	El atributo “work_type” se reemplaza con las siguientes reglas:
		“Private” → 0
		“Self-employed” → 1
		“Govt_job” → 2
		“children” → 3
		“Never_worked” → 4


Notar como al tratarse de arboles de decisión, no es necesario aplicar técnicas como “One Hot Encoding”  o “Bag Of Words”, pues el algoritmo no utiliza el valor especifico de un atributo sino que solo los utiliza para diferenciar valores con mismo valor en un atributo.
Es decir, el algoritmo no utiliza el dato que la instancia i tiene el valor “Private” y la instancia i’ tienen el valor “Self-employed”, sino que solo utiliza (para atributos discretos, que son los que se están transformando en este método) el hecho de que “Private” != “Self-employed”. 

Luego, esta codificación como enumerados es adecuada dada algunas simplificaciones menores como “no aparecera Govt_job y govtJob como valores distintos del atributo” y si se asume que los atributos categóricos contienen un numero acotado de valores “sin relación” (por ejemplo de distancia) entre ellos. Es decir, solo se opera comparando por igualdad.



## 2.4 División de conjunto de datos

Lo segundo que se hace en esta etapa es dividir el dataset en un conjunto de entrenamiento y otro de testeo, según los parámetros indicados en la letra y mediante el uso de la librería imblearn.


Posteriormente se calcula sobre el conjunto de entrenamiento, para cada atributo discreto, cual es el valor con mas ocurrencias en el dataset y para cada atributo continuo la mediana de dicho atributo.


Luego, tanto en el dataset de entrenamiento como en el de prueba, se reemplazan los valores nulos de los atributos con los valores anteriormente calculados. Particularmente, los atributos discretos toman el valor del atributo con mayor numero de ocurrencias y los continuos el valor de la mediana. Estos mismos valores son los que utilizará “TreeWrapper” para proveer la funcionalidad ya descripta.


Nuevamente, en virtud de la simplicidad y la reproducibilidad se decide no utilizar métodos estocásticos para asignarle valores a los atributos con valor nulo, si bien se podría calcular el porcentaje de veces que aparece cada valor de un atributo y generar valores para los nulos en base  a una distribución de probabilidad proporcional al numero de apariciones de dicho valor.


# 3. Experimentación

En esta sección se presentan los experimentos realizados sobre el algoritmo anteriormente presentado y el dataset de referencia. Se comenzará introduciendo algunas consideraciones las cuales serán base para todas las experimentaciones.




## 3.1 Consideraciones previas

Para cada experimento se entrenará un clasificador en base al algoritmo ID3 previamente implementado utilizando el conjunto de datos preprocesado como se menciona en la sección anterior. Se evaluarán los resultados utilizando las medidas de accuracy,   precision,   recall   y medida   F1, y se comparan estos resultados contra los obtenidos al entrenar un clasificador con la librería sklearn para el mismo conjunto de entrenamiento y de evaluación.

Para obtener las métricas antes mencionadas, se utilizaron las funciones  classification_report y confusion_matrix, ambas pertenecientes a la librería sklearn. En las columnas se encontrarán las métricas, mientras que en las filas las predicciones del clasificador. True representa una predicción positiva para un stroke y False una predicción negativa.

Dado que además de generar un clasificador queremos testearlo, lo que se hizo fue dividir el conjunto de datos original en dos subconjuntos disjuntos. El 80% de los datos fue destinado al conjunto de train mientras que el 20% restante se utilizó para el conjunto de test. Para generar estos subconjuntos se utilizó la función train_test_split perteneciente a la librería sklearn. Nótese que esta función separa un dataset en un conjunto de entrenamiento y un conjunto de datos de evaluación de manera random, por lo que para que el experimento sea reproducible lo que se hizo fue pasarle un entero constante al parámetro random_state de la función (el cual llamamos seed), lo que permite que siempre separe el set en los mismos subconjuntos de datos. El valor de SEED que utilizamos fue 27.
Esta misma seed fue utilizada en todas las funciones random de sklearn que fueron necesarias, de modo que todos los test sean reproducibles. 
		
A continuación se presentarán los diferentes experimentos realizados.


## 3.2 Entrenamiento 1

**Datos de prueba utilizados**

Para este experimento se utilizó el conjunto de entrenamiento y de evaluación sin ningún preprocesado previo aparte del mencionado en la sección anterior. 


In [ ]:
python .\main.py


Para el clasificador entrenado utilizando ID3, el resultado fue el siguiente:

**REPORTE**
<table>
<tr><td></td><td>Precision</td><td>Recall</td><td>F1-Score</td><td>Support</td></tr>
<tr><td>False</td><td> 0.96 </td><td> 0.95 </td><td> 0.96 </td><td> 974 </td></tr>
<tr><td>True</td><td> 0.15 </td><td> 0.17 </td><td> 0.16 </td><td> 48 </td></tr>
<tr><td>Accuracy</td><td>  </td><td>  </td><td> 0.92 </td><td>  </td></tr>
</table>

**MATRIZ DE CONFUSION**
<table>
<tr><td></td><td>h(x) = 0</td><td>h(x) = 1</td></tr>
<tr><td>y = 0</td><td> 930 </td><td> 44 </td></tr>
<tr><td>y = 1</td><td> 40 </td><td> 8 </td></tr>
</table>

**COMENTARIOS**

Los resultados obtenidos no fueron buenos ya que si bien los valores de precisión, exhaustividad y medida F1 fueron muy buenos para las predicciones de strokes negativas, al predecir casos positivos obtuvo poca precisión (precision= 0.15) y una baja exhaustividad (recall = 0.17). Estos parámetros se ven reflejados en la medida F1-Score con un valor de tan solo 0.16 para la detección de positivos.
Esto implica que el clasificador no es muy bueno cumpliendo su propósito de predecir casos positivos. 
Esto era esperable ya que partimos de un conjunto de datos desbalanceado, en el cual tenemos muchos datos para resultados negativos, pero pocos para resultados positivos.

Por otro lado vemos que el valor de accuracy es bastante bueno (accuracy = 0.92), pero claramente el clasificador tiene deficiencias. Esto ilustra los peligros de considerar la accuracy como unica medida de performance de un algoritmo y como tener un dataset desbalanceado puede perturbar dicha medida, causando problemas en la evaluacion.
Las deficiencias del algoritmo son sin duda relevantes. Particularmente, tomando como clase positiva la clase True, y considerando que esto se interpreta como la prediccion de que una persona tenga un ataque cerebrovascular, tener un recall de 0.17 indica que el algoritmo no es eficaz al detectar pacientes en peligro. De todas formas, dado el dataset de entrenamiento presentado, no es raro tener malos resultados en la clase positiva por el poco numero de ejemplos de dicha clase que existe respecto a la clase negativa.
Si bien en este contexto tampoco parecería ser un problema tan grave como el anterior, la precision (0.15) del algoritmo indica que tampoco es fiable al momento de predecir este tipo de enfermedad. Particularmente, se espera que mucha de la gente que clasifica como positiva sean falsos positivos.

Para el clasificador entrenado utilizando sklearn, el resultado fue el siguiente:

**REPORTE**
<table>
<tr><td></td><td>Precision</td><td>Recall</td><td>F1-Score</td><td>Support</td></tr>
<tr><td>False</td><td> 0.96 </td><td> 0.95 </td><td> 0.96 </td><td> 974 </td></tr>
<tr><td>True</td><td> 0.15 </td><td> 0.17 </td><td> 0.16 </td><td> 48 </td></tr>
<tr><td>Accuracy</td><td>  </td><td>  </td><td> 0.91 </td><td>  </td></tr>
</table>

**MATRIZ DE CONFUSION**
<table>
<tr><td></td><td>h(x) = 0</td><td>h(x) = 1</td></tr>
<tr><td>y = 0</td><td> 927 </td><td> 47 </td></tr>
<tr><td>y = 1</td><td> 40 </td><td> 8 </td></tr>
</table>

Al comparar los resultados obtenidos por ID3 contra los resultados presentados por sklearn, vemos que estos son muy similares. Esto reafirma un poco la idea de que el conjunto de datos con el que se entrenó puede no ser muy bueno, además da un poco de seguridad respecto a implementación del algoritmo presentada.

Como los resultados obtenidos no fueron buenos, se utilizaron las técnicas oversampling y undersampling para intentar mitigar el desbalance de los datos. 
La experimentación presentada a continuación se utilizó las implementaciones de oversampling y undersampling provistas por la librería sklearn. Para el caso del oversampling se realizó el sobremuestreo sobre el conjunto de datos minoritario (stroke positivos), y para la técnica de undersampling se realizó el submuestreo sobre el conjunto de datos mayoritarios (stroke negativos). 

Los siguientes análisis se concentran principalmente en las mediciones de la clase True, pues se obtuvieron buenos resultados para la clase False y estos se mantuvieron relativamente estables para la mayoria de los casos, solo se comentaran resultados sobre la clase Flase cuando existan cambios significativos.


## 3.3 Entrenamiento 2
**Datos de prueba utilizados:**

Para este experimento se utilizó el conjunto de entrenamiento el cual fue procesado previamente por la clase RandomOverSampler de sklearn.


In [ ]:
python .\main.py --random_oversampling 1 --unbalanced_dataset 0


Para el clasificador entrenado utilizando ID3, el resultado fue el siguiente:

**REPORTE**
<table>
<tr><td></td><td>Precision</td><td>Recall</td><td>F1-Score</td><td>Support</td></tr>
<tr><td>False</td><td> 0.96 </td><td> 0.96 </td><td> 0.96 </td><td> 974 </td></tr>
<tr><td>True</td><td> 0.20 </td><td> 0.23 </td><td> 0.22 </td><td> 48 </td></tr>
<tr><td>Accuracy</td><td>  </td><td>  </td><td> 0.92 </td><td>  </td></tr>
</table>

**MATRIZ DE CONFUSION**
<table>
<tr><td></td><td>h(x) = 0</td><td>h(x) = 1</td></tr>
<tr><td>y = 0</td><td> 931 </td><td> 43 </td></tr>
<tr><td>y = 1</td><td> 37 </td><td> 11 </td></tr>
</table>

**COMENTARIOS**

Comparando estos resultados con los obtenidos en el experimento 1, se puede notar una mejora en la precision, exhaustividad y medida F1, en particular la precisión aumentó un 5%, mientras que la exhaustividad y medida F1 aumentaron un 6%. Por otra parte la accuracy aumentó un 1% pasando a valer 92%.
Sin embargo, este aumento no es demasiado significativo y el clasificador sigue lejos de ser un clasificador el cual pueda ser considerado bueno, ya que como bien se observa en la matriz de confusión, este sigue dando muchos falsos positivos y falsos negativos. De todas formas, este método funcionó bastante bien considerando que se partió de un set altamente desbalanceado, por lo que se podria decir que fue eficaz dentro de lo que cabe.




## 3.4 Entrenamiento 3
Datos de prueba utilizados:

Para este experimento se utilizó el conjunto de entrenamiento el cual fue procesado previamente por la clase SMOTENC de sklearn.


In [ ]:
python .\main.py --smotenc_oversampling 1 --unbalanced_dataset 0


Para el clasificador entrenado utilizando ID3, el resultado fue el siguiente:

**REPORTE**
<table>
<tr><td></td><td>Precision</td><td>Recall</td><td>F1-Score</td><td>Support</td></tr>
<tr><td>False</td><td> 0.96 </td><td> 0.91 </td><td> 0.94 </td><td> 974 </td></tr>
<tr><td>True</td><td> 0.15 </td><td> 0.31 </td><td> 0.20 </td><td> 48 </td></tr>
<tr><td>Accuracy</td><td>  </td><td>  </td><td> 0.88 </td><td>  </td></tr>
</table>

**MATRIZ DE CONFUSION**
<table>
<tr><td></td><td>h(x) = 0</td><td>h(x) = 1</td></tr>
<tr><td>y = 0</td><td> 886 </td> <td> 88 </td></tr>
<tr><td>y = 1</td><td> 33 </td><td> 15 </td></tr>
</table>

**COMENTARIO**

Comparando estos resultados con los obtenidos en el experimento 1, se puede notar una mínima pérdida de precisión (de un 1%), sin embargo tanto la exhaustividad (recall) como la medida F1 aumentaron . En particular, la exhaustividad aumentó un 14% y la medida F1 un 6%. Esto se puede deber a que, como bien se ve en la matriz de confusión, hubo un aumento de predicciones positivas. Aunque esto mejora el clasificador en cierto sentido, esta mejora no es muy sustancial ya que aumenta la exhaustividad a costa de una pérdida de mantener una precisión constante prácticamente constante. Lo que genera que aumente la cantidad de falsos positivos que genera el clasificador.

Por otra parte la accuracy disminuyó un 3%.
Sin embargo, esto no es demasiado significativo y el clasificador sigue lejos de ser un clasificador el cual pueda ser considerado bueno, ya que como bien se observa en la matriz de confusión, este sigue dando muchos falsos positivos y falsos negativos. 

## 3.5 Entrenamiento 4
Datos de prueba utilizados:

Para este experimento se utilizó el conjunto de entrenamiento el cual fue procesado previamente por la clase RandomUnderSampler de sklearn.


In [ ]:
python .\main.py --undersampling 1 --unbalanced_dataset 0


Para el clasificador entrenado utilizando ID3, el resultado fue el siguiente:

**REPORTE**
<table>
<tr><td></td><td>Precision</td><td>Recall</td><td>F1-Score</td><td>Support</td></tr>
<tr><td>False</td><td> 0.98 </td><td> 0.69 </td><td> 0.91 </td><td> 974 </td></tr>
<tr><td>True</td><td> 0.09 </td><td> 0.65 </td><td> 0.16 </td><td> 48 </td></tr>
<tr><td>Accuracy</td><td>  </td><td>  </td><td> 0.69 </td><td>  </td></tr>
</table>

**MATRIZ DE CONFUSION**
<table>
<tr><td></td><td>h(x) = 0</td><td>h(x) = 1</td></tr>
<tr><td>y = 0</td><td> 672 </td><td> 302 </td></tr>
<tr><td>y = 1</td><td> 17 </td><td> 31 </td></tr>
</table>

**COMENTARIO**:

Comparando estos resultados con los obtenidos en el experimento 1, se puede notar una pérdida significativa de precisión (de un 6%), mientras que la exhaustividad aumentó un 48% y la medida F1 se mantuvo constante (para las predicciones TRUE).
Este gran aumento de la exhaustividad se debe a que, como bien se ve en la matriz de confusión, hubo un aumento de predicciones acertadas en positivos (31 de 48). Sin embargo, esto implicó una pérdida importante de precisión dado que el clasificador tuvo un aumento importante en la cantidad de predicciones falsamente positivas.

Para las predicciones de False, hubo un leve aumento de precisión (2%), mientras que se dio una pérdida significativa de exhaustividad (un 27%) y por ende en la medida F1.

Por otra parte, la accuracy disminuyó un 23% lo que da a entender que este método, aplicado sobre el conjunto de datos ya presentado, no da un buen resultado al ser utilizado por el algoritmo implementado.

# 4. Conclusión

## 4.1 Resultados según expectativas iniciales
Inicialmente, las expectativas con respecto al algoritmo no eran altas. Confirmando esta suposición, al realizar una evaluación estadística de los resultados se obtiene un valor de F1 de 0.16, considerado negativo por el equipo dado que la precisión a la hora de predecir valores es menor al 20%. Aún así, al comparar estos resultados con los logrados por la implementación de sklearn del árbol de decisión en sus múltiples variaciones se observan salidas casi idénticas en comparación a la implementación de la tarea, superando las expectativas iniciales.


En cuanto al desbalance del conjunto de datos, inicialmente se esperaba una mejora notoria en la recuperación. Sin embargo, los cambios son considerados casi imperceptibles, obteniendo valores ligeramente mayores en cuanto a recuperación al realizar técnicas de oversampling. Por el contrario, las pruebas con undersampling resultaron inesperadamente desfavorables con respecto a los datos iniciales.

## 4.2 ¿Cuándo se dieron los mejores resultados?
Los mejores resultados del algoritmo se dieron al intentar clasificar registros negativos, lo cual probablemente sea consecuencia del gran volumen de valores positivos en el conjunto de datos. Por otro lado, esta falencia es mitigada con la ayuda del oversampling, que mejoraba los resultados aunque sin solventar los problemas de desbalance por completo. En particular, la variación de smote permitió obtener la mayor recuperación de todas las variaciones probadas.

## 4.3 ¿Cómo se mejorarían los resultados?
Dados los resultados de la evaluación del algoritmo, es notoria la dificultad del algoritmo a la hora de clasificar instancias positivas. Esto llevado a un caso de la vida real podría costarle la vida a muchas personas, dado que de acuerdo con los resultados de aquellos que tendrán un derrame cerebral (stroke), no más de un 25% será diagnosticado correctamente. Por esto último se concluye que para este caso particular el algoritmo utilizado en este conjunto de datos está lejos de ser considerado útil aún con las mejoras planteadas, aunque podría ser considerado como posible solución para problemas menos críticos.


Por otro lado, un mejor conjunto de datos sin duda sería de ayuda, dado que su desbalance podría estar causando que el algoritmo simplemente puntúe como negativa a la mayoría de las entradas sin sentido aparente, dado que esto es lo que maximiza la tasa de aciertos. Aún así, si se utilizara un método de balanceo más efectivo existe la posibilidad de que los efectos negativos de esta característica sean resueltos. Adicionalmente, realizar un estudio probabilístico o hasta teórico de los atributos del conjunto de datos podría suponer una importante mejora tanto en el tiempo de ejecución del algoritmo como de la certeza de las predicciones. 

Además, en caso de haber elegido la codificación de atributos categóricos con One Hot Encoding, se podría haber experimentado con otros métodos de oversampling y undersampling provistas por imblearn. Dichas técnicas se evitaron pues eran de alguna forma susceptibles a la distancia entre los vectores y la codificación basada en enteros da un orden y distancia implícitos que no necesariamente son significativos. No obstante, dado el algoritmo particular, no parece una mala opción la presentada, dada la simplicidad que presenta, sacrificando en nuestro caso potencial de experimentar con más técnicas de balance del dataset más complejas.

Por otra parte, podría ser positivo sustituir los valores desconocidos (e.g. NaN o Unknown) con un mecanismo que saque provecho del algoritmo implementado. En particular, Mitchell propone técnicas que se aprovechan de la característica inductiva del árbol sustituyendo estos valores de acuerdo a la probabilidad de máxima verosimilitud del conjunto de datos asociado a cada nodo en particular.


Por último, en cuanto a la arquitectura, sería conveniente para un futuro uso del programa que se obtengan los valores de las columnas al igual que su dominio de forma dinámica para adaptar el problema a distintos conjuntos de datos y hasta problemas de predicción congruentes.

# 5. Referencias

[1] Mitchell – Mitchell, T. M. (1994). Machine Learning. (pg. 56)

[2] Mitchell – Mitchell, T. M. (1994). Machine Learning. (pg. 72-73)

[3] Mitchell – Mitchell, T. M. (1994). Machine Learning. (pg. 69)